# 00 – Ingest Bronze
Fill placeholders via widgets or Job params.

In [0]:
dbutils.widgets.text("CATALOG","reporting_factory_risk_profile")
dbutils.widgets.text("RAW_PATH","abfss://raw@riskprofilestorageacct.dfs.core.windows.net/lending_club/")
catalog = dbutils.widgets.get("CATALOG")
raw = dbutils.widgets.get("RAW_PATH")

key = dbutils.secrets.get(scope="adls-riskprofile-secrets", key="adls-riskprofile-key")
spark.conf.set(
  "fs.azure.account.key.riskprofilestorageacct.dfs.core.windows.net",
  key
)

spark.sql(f"USE CATALOG {catalog}")
spark.sql("USE SCHEMA bronze")

df = (spark.read
      .option("header", True)
      .option("inferSchema", True)
      .csv(f"{raw}*.csv"))

(df.write
   .mode("overwrite")
   .format("delta")
   .saveAsTable("bronze.lending_raw"))

print(f"Bronze loaded: {catalog}.bronze.lending_raw")
